In [ ]:
!pip install unstructured
!pip install unstructured[pdf]
!pip install -U langchain-openai
!pip install chromadb
!pip install pypdf
!pip install tqdm
!pip install sentence-transformers
!pip install transformers
!pip install InstructorEmbedding

In [ ]:
!pip install pinecone-client
!pip install langchain

In [ ]:
!pip install transformers datasets spacy scikit-learn
!python -m spacy download en_core_web_sm

In [ ]:
! pip install pyngrok
! pip install flask-ngrok
! ngrok authtoken 2fgafRv4DMXaDwep9sSlHpsPgeB_76vC81WKH5ns7owcmDeWC
! pip install streamlit

In [138]:
!pip install cmake
!pip install llama-cpp-python \
  --extra-index-url https://abetlen.github.io/llama-cpp-python/whl/cu121

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 48.9 MB/s eta 0:00:00:00:0100:01
Looking in indexes: https://pypi.org/simple, https://abetlen.github.io/llama-cpp-python/whl/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.8/141.8 MB 10.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 784.9 kB/s eta 0:00:000:00:01


In [198]:
%%writefile fastllm.py
import os
import urllib.request
from llama_cpp import Llama

def download_file(file_link, filename):
    # Checks if the file already exists before downloading
    if not os.path.isfile(filename):
        urllib.request.urlretrieve(file_link, filename)
        print("File downloaded successfully.")
    else:
        print("File already exists.")


# Dowloading GGML model from HuggingFace
ggml_model_path = "https://huggingface.co/TheBloke/zephyr-7B-beta-GGUF/resolve/main/zephyr-7b-beta.Q4_0.gguf"
filename = "zephyr-7b-beta.Q4_0.gguf"

download_file(ggml_model_path, filename)

from llama_cpp import Llama

model_path='zephyr-7b-beta.Q4_0.gguf'
llm = Llama(model_path=model_path,
            n_gpu_layers=50,n_ctx=10000)

Overwriting fastllm.py


In [156]:
import os
os.makedirs('/kaggle/working/pages')

In [232]:
os.remove('/kaggle/working/our_llm.py')

In [ ]:
!pip install streamlit-extras

In [241]:
%%writefile Homepage.py
import streamlit as st

st.set_page_config(
    page_title="Resume Improver",
    page_icon="📜",
)

st.write("Welcome to LLM powered resume Improver")

st.sidebar.success("Select what you would like to do")

st.markdown(
    """
    This resume improver is based on mistral-7b LLM. It has 2 features, namely Multiple Resume Comparison
    and Resume And Job Description. Multiple Resume Comparison gives information about how the submitted resumes 
    differe from one another and which one is better. Resume And Job Description on the other hand requires one job 
    description and one resume which is to be improved on the basis of that Job Description. 
"""
)
if 'documents' not in st.session_state:
    st.session_state['documents'] = None


Overwriting Homepage.py


In [242]:
%%writefile pages/1_Multiple_Resume_Comparison.py
import streamlit as st
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import Chroma
from langchain_community.document_loaders import UnstructuredPDFLoader
from fastllm import llm
from langchain.text_splitter import RecursiveCharacterTextSplitter

persist_directory = 'db'
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")


st.set_page_config(
    page_title="Multi Resume",
    page_icon="📜"

)

# Sidebar for uploading PDF files
docs=[]
with st.sidebar:
    st.title("Resume Comparison:")
    pdf_docs = st.file_uploader("Upload your resumes in the form of PDFs ", accept_multiple_files=True)
    if pdf_docs is not None:
        with st.spinner("Processing..."):
            for i,file in enumerate(pdf_docs):
                name="resume_file"+str(i)+".pdf"
                with open(name, 'wb') as f:
                    f.write(file.getvalue())
                path="/kaggle/working/"+name
                loader = UnstructuredPDFLoader(path)
                data = loader.load()
                docs.append(data[0]) 
                #st.session_state['documents'] = docs
            st.success("Done")
            
#docs = st.session_state['documents']
if len(docs)>0:
    vectordb = Chroma.from_documents(documents=docs, embedding=embeddings, persist_directory=persist_directory)
    vectordb.persist()
    vectordb = None
    vectordb = Chroma(persist_directory=persist_directory,embedding_function=embeddings)                
    retriever = vectordb.as_retriever(search_kwargs={"k": 5})
    query = st.text_input("What are you looking for in these candidates?")
    if not query:
        st.stop()
    docs = retriever.invoke(query)

    def split_docs(documents, chunk_size=1000, chunk_overlap=100):
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
        docs = text_splitter.split_documents(documents)
        return docs

    docs=split_docs(docs)

    prompt="Following is the conversation between the User and AI. AI is a hiring assistant chatbot.AI give short and very precise answers. AI's task is to answer the question based on the following resumes :"

    for i in range(5):
        prompt=prompt+"\n"+f"{i+1}th Resume : "+"\n"+docs[i].page_content

    history=prompt
    input_text=""
    n=0
    while query!="Quit" or query!="quit":
        if n==0:
            input_text=f"Please explain why the said above resumes are more relevant with query : {query} "
        else:
            input_text = st.text_input("What more would you like to ask?",key=n)
            if not input_text:
                st.stop()

        if input_text=="quit":
            break

        history=history+ f"User : {input_text}" + "\n" + "AI :"
        response=llm(history, max_tokens=300)

        out=response["choices"][0]["text"].strip()
        lines = history.split('\n')


        new_last_line = f"AI : {out}"
        n=n+1
        lines[-1] = new_last_line
        history='\n'.join(lines)
        i=history.rfind("AI:")
        st.write(out)


Overwriting pages/1_Multiple_Resume_Comparison.py


In [243]:
%%writefile pages/2_Resume_And_Job_Description.py
import streamlit as st
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import Chroma
from langchain_community.document_loaders import UnstructuredPDFLoader
from fastllm import llm

persist_directory = 'db'
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")


st.set_page_config(
    page_title="Job Description",
    page_icon="📜"
)

# Sidebar for uploading PDF files
docs=[]
with st.sidebar:            
    st.title("Resume And Job Description:")
    resume = st.file_uploader("Upload your resume ", accept_multiple_files=False)
    jd = st.file_uploader("Upload your Job description ", accept_multiple_files=False)
    if resume is None or jd is None:
        st.stop()
    with st.spinner("Processing..."):

            with open("resume_file.pdf", 'wb') as f:
                f.write(resume.getvalue())

            with open("jd_file.pdf", 'wb') as f:
                f.write(jd.getvalue())

            resume_path="/kaggle/working/resume_file.pdf"
            jd_path="/kaggle/working/jd_file.pdf"

            resume_loader = UnstructuredPDFLoader(resume_path)
            resume_data = resume_loader.load()
            resume_data = resume_data[0].page_content

            jd_loader = UnstructuredPDFLoader(jd_path)
            jd_data = jd_loader.load()
            jd_data = jd_data[0].page_content 
            st.success("Done")
if len(resume_data)>1000:
    resume_data=resume_data[0:1000]
prompt="Following is the conversation between the User and AI. AI is a hiring assistant that suggests improvements in resume accorind to the job description. This time use the following job description:"
prompt=prompt+jd_data
history=prompt
input_text="Please suggest improvements in this resume:"+resume_data
history=history+ f"User : {input_text}" + "\n" + "AI :"
response=llm(history, max_tokens=300)

out=response["choices"][0]["text"].strip()
lines = history.split('\n')


new_last_line = f"AI : {out}"

lines[-1] = new_last_line
history='\n'.join(lines)
i=history.rfind("AI:")
st.write(out)

Overwriting pages/2_Resume_And_Job_Description.py


In [ ]:
from pyngrok import ngrok
ngrok.set_auth_token("2fgafRv4DMXaDwep9sSlHpsPgeB_76vC81WKH5ns7owcmDeWC")

public_url =  ngrok.connect(80).public_url
print(f"Please click on the text below {public_url}")
!streamlit run --server.port 80 Homepage.py > /dev/null

Please click on the text below https://e6f9-35-223-73-195.ngrok-free.app
/opt/conda/lib/python3.10/site-packages/langchain/embeddings/__init__.py:29: LangChainDeprecationWarning: Importing embeddings from langchain is deprecated. Importing from langchain will no longer be supported as of langchain==0.2.0. Please import from langchain-community instead:

`from langchain_community.embeddings import SentenceTransformerEmbeddings`.

To install langchain-community run `pip install -U langchain-community`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/langchain/vectorstores/__init__.py:35: LangChainDeprecationWarning: Importing vector stores from langchain is deprecated. Importing from langchain will no longer be supported as of langchain==0.2.0. Please import from langchain-community instead:

`from langchain_community.vectorstores import Chroma`.

To install langchain-community run `pip install -U langchain-community`.
  warnings.warn(
llama_model_loader: loaded meta data with 2